![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BertForZeroShotClassification.ipynb)

## Import ONNX RoBertaForZeroShotClassification  models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `RoBertaForZeroShotClassification ` is only available since in `Spark NLP 5.2.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for zero shot classification via `RoBertaForZeroShotClassification` or `TFRoBertaForZeroShotClassification`. These models are usually under `Zero-Shot Classification` category and have `roberta` in their labels
- Reference: [TFRoBertaForZeroShotClassification](https://huggingface.co/docs/transformers/en/model_doc/roberta#transformers.TFRobertaForSequenceClassification)
- Some [example models](https://huggingface.co/models?pipeline_tag=zero-shot-classification&sort=downloads&search=roberta)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.34.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.39.3 optimum sentencepiece tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 106.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [cross-encoder/nli-roberta-base](https://huggingface.co/cross-encoder/nli-roberta-base) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.

In [2]:
from optimum.onnxruntime import ORTModelForSequenceClassification
import tensorflow as tf

MODEL_NAME = 'cross-encoder/nli-roberta-base'
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Framework not specified. Using pt to export the model.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

***** Exporting submodel 1/1: RobertaForSequenceClassification *****
Using framework PyTorch: 2.3.1+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 490488
-rw-r--r-- 1 root root       883 Aug  4 11:15 config.json
-rw-r--r-- 1 root root    456318 Aug  4 11:15 merges.txt
-rw-r--r-- 1 root root 498873677 Aug  4 11:15 model.onnx
-rw-r--r-- 1 root root       957 Aug  4 11:15 special_tokens_map.json
-rw-r--r-- 1 root root      1214 Aug  4 11:15 tokenizer_config.json
-rw-r--r-- 1 root root   2108614 Aug  4 11:15 tokenizer.json
-rw-r--r-- 1 root root    798293 Aug  4 11:15 vocab.json


- As you can see, we need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [4]:
!mkdir {EXPORT_PATH}/assets

In [5]:
# get label2id dictionary
labels = ort_model.config.id2label
# sort the dictionary based on the id
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))


In [12]:
import json
output_json = json.load(open(f"{EXPORT_PATH}/vocab.json"))

with open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as f:
    for key in output_json.keys():
        print(key, file=f)

!cp {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets


Voila! We have our `vocab.txt` and `merges.txt` and `labels.txt` inside assets directory

In [13]:
!ls -lR {EXPORT_PATH}

onnx_models/cross-encoder/nli-roberta-base:
total 490492
drwxr-xr-x 2 root root      4096 Aug  4 11:31 assets
-rw-r--r-- 1 root root       883 Aug  4 11:15 config.json
-rw-r--r-- 1 root root    456318 Aug  4 11:15 merges.txt
-rw-r--r-- 1 root root 498873677 Aug  4 11:15 model.onnx
-rw-r--r-- 1 root root       957 Aug  4 11:15 special_tokens_map.json
-rw-r--r-- 1 root root      1214 Aug  4 11:15 tokenizer_config.json
-rw-r--r-- 1 root root   2108614 Aug  4 11:15 tokenizer.json
-rw-r--r-- 1 root root    798293 Aug  4 11:15 vocab.json

onnx_models/cross-encoder/nli-roberta-base/assets:
total 852
-rw-r--r-- 1 root root     32 Aug  4 11:24 labels.txt
-rw-r--r-- 1 root root 456318 Aug  4 11:31 merges.txt
-rw-r--r-- 1 root root 407065 Aug  4 11:31 vocab.txt


## Import and Save RoBertaForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-09-29 19:41:03--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-09-29 19:41:04--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-09-29 19:41:04 (106 MB/s) - written to stdout [1191/1191]

Installing 

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `RoBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

zero_shot_classifier = RoBertaForZeroShotClassification.loadSavedModel(
    EXPORT_PATH,
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your RoBertaForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 429464
-rw-r--r-- 1 root root 439759046 Sep 29 19:42 bert_classification_onnx
drwxr-xr-x 4 root root      4096 Sep 29 19:42 fields
drwxr-xr-x 2 root root      4096 Sep 29 19:42 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForZeroShotClassification model 😊

In [ ]:
zero_shot_classifier_loaded = RoBertaForZeroShotClassification.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
zero_shot_classifier_loaded.getClasses()

['NEU', 'POS', 'NEG']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

text = [["I have a problem with my iphone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to harry up and get to the airport. We are going to miss our flight!"]]

# create a DataFrame in PySpark
inputDataset = spark.createDataFrame(text, ["text"])
model = pipeline.fit(inputDataset)
model.transform(inputDataset).select("class.result").show()

+------------------+------+
|              text|result|
+------------------+------+
|Te quiero. Te amo.| [POS]|
+------------------+------+



That's it! You can now go wild and use hundreds of `RoBertaForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
